In [2]:
import numpy as np
import tensorflow as tf
from sympy import *
import re
import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox
from matplotlib.widgets import Button
import ipywidgets as widgets

In [4]:
plt.close("all")